In [163]:
import bs4
import datetime as dt
import pandas as pd
import random 
import requests
from bs4 import BeautifulSoup as bs
from tqdm import tqdm 

## Генерация номеров телефонов

In [78]:
def get_data(element: bs4.element.ResultSet, name_for_column: str = None)->pd.DataFrame():
    l = []
    for value in element:
        l.append(value.contents[0])
    l.pop(0)
    values  = pd.DataFrame(l)
    if name_for_column != None:
        values = values.rename(columns={0:name_for_column})
    return values

def get_codec()->pd.DataFrame():
    URL_TEMPLATE = "https://www.ixbt.com/mobile/country_code.html"
    r = requests.get(URL_TEMPLATE)
    if r.status_code != 200:
        raise Exception("Site not working.")
    soup = bs(r.text, "html.parser", )
    code = soup.find('table', border=1, cellspacing=2, cellpadding=2, width=550, align="center")
    names, codes = soup.find_all("td", width="128"), soup.find_all("td", width="148")
    names = get_data(names, "country")
    codes = get_data(codes, "code")
    
    return pd.concat([names, codes], axis=1, sort=False)

In [79]:
data = get_codec()
data["check"] = data.code.str.find("-")    
data = data.query("check == -1")    
data["check"] = data.code.isnull()
data = data.query("check == False")    

In [227]:
numbers = []
for i in tqdm(range(5000)):
    region_code = random.randint(100, 999)
    first_three_digits = random.randint(100, 999)
    last_four_digits = random.randint(1000, 9999)
    index = random.randint(0, len(data)-1)
    index = data.iat[index, 1]
    numbers.append(f"{index} {region_code}-{first_three_digits}-{last_four_digits}".strip())

100%|███████████████████████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 38324.38it/s]


In [228]:
numbers = pd.DataFrame(numbers)

In [229]:
numbers = numbers.rename(columns={0:"numbers"})

In [230]:
numbers_fraud = numbers.sample(1000).reset_index()

In [231]:
numbers = numbers.sample(len(numbers)).reset_index()

## Данные о сессиях

In [232]:
date_start = dt.datetime.strptime('01.01.2022', "%d.%m.%Y")
delta = (dt.datetime.now() - date_start).days

In [233]:
date_st = [(date_start + dt.timedelta(days=random.randint(-100, delta))) for i in range(0, len(numbers_fraud))]
date_end = [(date_start + dt.timedelta(days=random.randint(-100, delta))) for i in range(0, len(numbers_fraud))]

In [234]:
numbers_fraud = pd.concat([numbers_fraud,
          pd.DataFrame(date_st, columns=['Date_start']),
          pd.DataFrame(date_end, columns=['Date_end'])], axis=1)

reset = numbers_fraud.query("Date_start > Date_end")
numbers_fraud = numbers_fraud.query("Date_start <= Date_end")

reset = reset.rename(columns={'Date_start': 'Date_end', 'Date_end': 'Date_start'})
reset = reset[list(numbers_fraud.columns)]

numbers_fraud = pd.concat([numbers_fraud,
          reset], axis=0)

for i in tqdm(numbers_fraud.index):
    numbers_fraud.at[i, 'index'] = random.randint(0, 3000)

100%|███████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 47294.40it/s]


In [235]:
numbers_fraud.query("Date_start > Date_end")

,index,numbers,Date_start,Date_end


In [236]:
date_st = [(date_start + dt.timedelta(days=random.randint(-100, delta))) for i in range(0, len(numbers))]
date_end = [(date_start + dt.timedelta(days=random.randint(-100, delta))) for i in range(0, len(numbers))]

In [237]:
numbers = pd.concat([numbers,
          pd.DataFrame(date_st, columns=['Date_start']),
          pd.DataFrame(date_end, columns=['Date_end'])], axis=1)

reset = numbers.query("Date_start > Date_end")
numbers = numbers.query("Date_start <= Date_end")

reset = reset.rename(columns={'Date_start': 'Date_end', 'Date_end': 'Date_start'})
reset = reset[list(numbers.columns)]

numbers = pd.concat([numbers,
          reset], axis=0)

for i in tqdm(numbers.index):
    numbers.at[i, 'index'] = random.randint(0, 3000)

100%|███████████████████████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 76152.90it/s]


In [238]:
numbers.query("Date_start > Date_end")

,index,numbers,Date_start,Date_end


In [239]:
numbers_fraud

,index,numbers,Date_start,Date_end
0,328,+63 405-978-4153,2021-10-17,2023-01-24
1,1992,+221 554-814-7187,2022-02-16,2023-03-24
2,1772,+30 184-676-7886,2022-12-31,2023-01-19
4,1772,+594 795-609-1527,2022-09-07,2023-06-16
5,172,+670 303-668-9342,2021-10-25,2022-01-28
...,...,...,...,...
993,1552,+351 925-159-9482,2023-05-15,2023-06-10
995,2312,+688 659-261-7045,2022-12-07,2022-12-18
996,659,+266 847-121-3481,2022-01-22,2022-06-30
997,2381,+358 460-413-4332,2022-10-11,2023-05-21


In [240]:
numbers

,index,numbers,Date_start,Date_end
4,574,+257 452-368-6590,2022-02-19,2022-08-28
6,1867,+593 805-142-5345,2022-05-26,2023-06-03
8,1724,+387 237-958-6655,2022-10-05,2023-05-07
11,1026,+967 413-658-2278,2022-06-06,2022-12-22
12,721,+44 179-209-3473,2022-06-21,2022-12-22
...,...,...,...,...
4986,462,+374 756-919-1155,2021-10-21,2023-05-22
4988,2641,+212 606-384-7220,2022-08-27,2022-10-06
4991,2991,+82 883-850-8956,2022-07-12,2023-03-01
4995,1217,+672 737-719-7539,2022-08-14,2023-05-31


In [241]:
numbers.to_excel('./Сессии.xlsx', index=False)
numbers_fraud.to_excel('./Мошенники.xlsx', index=False)